# On the topic of fields, semantics, and probabilistic results





While working with the Cicada library, it is important to keep in mind that the values you see and work with which often go through the steps of encoding and decoding do not actually exist when they are being manipulated in a series of secret shared operations. Our fixed-point arithmetic system is a mapping of a finite number of elements with fixed binary precision onto a finite number of elements which are simply a set of integers modulo some prime large enough to accommodate the range of interest. To see this directly, inspect the encoding and decoding functions for yourself. A link is given here for convenience: 

https://github.com/cicada-mpc/cicada-mpc/blob/main/cicada/encoder/fixedfield.py

The functions do assume that the encoder has already been instantiated and its parameters defined such as scale which is simply 2^precision where precision is the number of fractional bits, or the number of bits to the right of the radix point. Any fractional precision requiring more bits to represent present in the value to be encoded will be lost via the encoding process. Also note that if a value is negative it will fall into the upper half of the field due to the application of the modulus. This is made more explicit by the decoding function which relies on some additional variables which are assigned at the time of the encoder’s instantiation, specifically, “posbound” which is equal to the floor of the prime modulus halved. Posbound is the threshold in the field over which a value will be interpreted as negative. Thus, the field is split into two regions the upper half of the field is dedicated to representing negative numbers while the lower half of the field represents positive numbers. Let's go ahead an instantiate a small one for examples. 


Let us consider an 8 bit field. The largest 8 bit prime is $251=2^8-5$. That indicates that posbound will be 125, meaning all values in the range [126,250] will be interpreted as negative. In our example encoder we’ll have 3 bits of fractional precision. This means the scale used in encoding and decoding will be $2^3=8$. Notice the arguments given to the protocol constructor below, especially the 251 for the modulus to be used for the encoder, and the 3 to be used for the fractional part bitwidth.

In [ ]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

In [ ]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, None, None, 251, 3)
    
    log.info(f"Player {communicator.rank} protocol encoder info of interest:\n\tModulus: \t\t\t{protocol.encoder.modulus}\n\tBitwidth: \t\t\t{protocol.encoder.fieldbits}\n\tPrecision(fractional bits): \t{protocol.encoder.precision}\n\tScale: \t\t\t\t{protocol.encoder._scale}\n\tPosbound: \t\t\t{protocol.encoder._posbound}", src=1)    

main();

These issues have a few follow on implications. The first is that overflow and underflow can and will happen without any notice if code is not written carefully since all the operations occur obliviously. Take care that the operations are applied to operands with no or very small chance of this happening. It may happen in unexpected situations and yield similarly unexpected results. For example, the addition of two positive values may yield a negative seemingly non-sensical result if their sum puts them into the upper half of the field which will later be decoded as a negative value.

Secondly, division is not directly possible in the context of the field since it is an integral field no notion of values less than one exists in that context. We use field elements to represent fractional values, but these are somewhat bootstrapped on semantics that have no significance to the field itself. We can get a stable and expected result of division by multiplying by an elements multiplicative inverse in the context of the field, but this has the desired result iff the intended dividend has the desired divisor as a factor. Otherwise the result will not yield any useful value for the external semantics. In general, we perform division via approximation, masking, and the like. The accuracy of the result from any division operation is heavily dependent on the precision available from the encoder with respect to the number of bits right of the radix.  

Next let's try some examples. For each of the following we will provide the exmaple, work it out "by hand" and then show the example as implemented in cicada. 

* Encode then decode the value 3.25 in the field given the described encoder: 
    * We multiply by the scale (in this case $2^3$) $3.25\cdot8=26$ This is positive and less than the modulus so there are no concerns there; we are done.
    * To decode we check if the value (26) is greater than posbound (it isn’t) so we divide by the scale and return the value $26/8=3.25$


In [ ]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, None, None, 251, 3)
    val = numpy.array(3.25)
    enc_val = protocol.encoder.encode(val)
    dec_val = protocol.encoder.decode(enc_val)
    log.info(f"Player {communicator.rank} \n\tValue: \t\t{val}\n\tValue Encoded:\t{enc_val}\n\tValue Decoded:\t{dec_val}", src=1)    

main();

* Encode then decode the value -3.25 in the field given the described encoder: 
    * We multiply by the scale (in this case 2^3) -3.25*8=-26 This is negative so we apply the modulus i.e., -26 mod 251 = 225.
    * To decode we check if the value (225) is greater than posbound (it is) so we compute the additive inverse of the difference between the modulus and the value i.e., -(251-225)=-26, then divide by the scale and return the value -26/8=-3.25


In [ ]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, None, None, 251, 3)
    val = numpy.array(-3.25)
    enc_val = protocol.encoder.encode(val)
    dec_val = protocol.encoder.decode(enc_val)
    log.info(f"Player {communicator.rank} \n\tValue: \t\t{val}\n\tValue Encoded:\t{enc_val}\n\tValue Decoded:\t{dec_val}", src=1)    

main();

* Encode then decode the value 3.0625 in the field given the described encoder: 
    * We multiply by the scale 3.0625*8=24.5 This is positive and less than the modulus, but not an integral value so we truncate to 24. We are done.
    * To decode we check if the value (24) is greater than posbound (it isn’t) so we divide by the scale and return the value 24/8=3
    * Checking against the original value it is clear to see that we have lost the fractional part of the original (0.0625). This is due to the fact that in binary it is represented as 0.0001 and we have only 3 bits of binary precision available. Specifically, this happened at the point we truncated 24.5 to 24 which is a necessary step to make sure every value is both consistent in semantics and compatible with representation in our integral field. 


In [ ]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, None, None, 251, 3)
    val = numpy.array(3.0625)
    enc_val = protocol.encoder.encode(val)
    dec_val = protocol.encoder.decode(enc_val)
    log.info(f"Player {communicator.rank} \n\tValue: \t\t{val}\n\tValue Encoded:\t{enc_val}\n\tValue Decoded:\t{dec_val}", src=1)    

main();

* Encode then add 15 and 2 in the field and finally decode given the described encoder:
    * In a similar manner to the preceding, the encoding of 15 and 2 is 120 and 16 respectively. 
    * The sum of these is 136
    * Decoding 136 yields -14.375, not the answer we were expecting as the sum of 15 and 2, due to overflow of the representable positive range in our semantic mapping onto the field. In practice much larger fields are used so that incidents such as this are far easier to avoid. For example a 64 bit field is used in cicada by default though even larger ones can be optionally instantiated.


In [ ]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, None, None, 251, 3)
    val1 = numpy.array(15)
    val2 = numpy.array(2)
    enc_val1 = protocol.encoder.encode(val1)
    enc_val2 = protocol.encoder.encode(val2)
    
    sumd_vals = protocol.encoder.add(enc_val1, enc_val2) 
    dec_sumd_vals = protocol.encoder.decode(sumd_vals)
    log.info(f"Player {communicator.rank} \n\tValue 1: \t\t{val1}\n\tValue 2: \t\t{val2}\n\tValue 1 Encoded:\t{enc_val1}\n\tValue 2 Encoded:\t{enc_val2}\n\tSum Encoded:\t\t{sumd_vals}\n\tValue Decoded:\t\t{dec_sumd_vals}", src=1)    

main();

Another area of concern wrt such issues are the max and min functions. Given the sematic meaning we are mapping onto the field, at the border of this mapping some problems may arise. Our implementation of these functinoalities is based on the following algebraic expressions:

$$max(x, y)=(x+y+abs(x-y))/2$$

$$min(x, y)=(x+y+abs(x-y))/2$$

which will behave as expected much of the time, however, if the difference between $x$ and $y$ wraps around an end of the field more than once then problems can arise. Given a field $\mathbb{Z}_p$ as long as the operands are either both of the same sign, or, both satisfy the (in our opinion reasonable) condition/expectation that $abs(x)<p//4$ then the function should behave as anticipated.

Finally, on the topic of probabilistic results there are some functionalities we have implemented which may be off from the true answer by some relatively negligible margin. For example, consider the approach we have implemented for truncation. The function masks a shared secret with a random field element. It is possible that the sum of the operand and the random field element will overflow the field, which would cause the whole process to be wildly inaccurate. This is very unliekly to happen in practice however if the operands are relatively small in comparison to the size of the field. Additionally, a small error (overflow in the least significant bits generated from a carry coming into this region from the bits that were truncated) is also a possibility. We will now demonstrate an instance of this occurence presently:

In [ ]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)#, None, None, 251, 3)
    
    test_val = numpy.array(65536.5)
    shared_val = protocol.share(src=0, secret=protocol.encoder.encode(test_val), shape=test_val.shape)
    truncd_val = 1.0000152587890625
    counter = 0
    right_count = 0
    err_sum = 0
    while counter <100:     
        truncd_shared_val = protocol.truncate(shared_val)    
        truncd_val = protocol.encoder.decode(protocol.reveal(truncd_shared_val))
        counter += 1
        #log.info(f'attempt {counter}', src=0)
        if truncd_val == 1:
            right_count += 1
        else:
            err_sum += abs(truncd_val-1)
    log.info(f"Player {communicator.rank} \n\tTest Value: \t\t{test_val}\n\tTruncd Value: \t\t{truncd_val}\n\tExpected Value:\t\t{1.0}\n\tPercent Correct: \t{right_count}\n\tAverage Error:\t\t{err_sum/(100-right_count)}\n\tAverage answer:\t\t{1*(right_count/100)+(1+err_sum/(100-right_count))*(100-right_count)/100}", src=1)    
        
    

main();

This has just performed 100 truncations on a value which is specifically selected to given an error probability of 50%. Most probably the Percent correct reported above is somewhere between 45 and 55, and the average error is probably less than 2e-5. This is caused by specifically setting the most significant bit in the region to be truncated thereby making it a roughly 50% probability based on the underlying random bit generation protocol that the corresponding bit in the mask will be set which in turn generates a cary into the region which will be unaffected by the truncation yielding the relatively insignificant error we see.